In [12]:
# Source - https://stackoverflow.com/a
# Posted by pv., modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-06, License - CC BY-SA 3.0

%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt

from cb25d.batch import run_batch_simulation
from cb25d.compare_gamma_original import run_gamma_comparison_original
from cb25d.interactive import run_interactive_simulation
from cb25d.notebook import init, savefig
import time

from cb25d.interactive import run_interactive_simulation
from cb25d.simulation_impl_ext import (
    generate_extended_initial_conditions, 
    SimulationRecorderExtended
)

# np.random.seed(42) # For reproducibility ?



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Comparing to the 'original' model by approximating its behavior
initial_state_ext = generate_extended_initial_conditions(
    # copied from 'original' model parameters
    seed=1,
    n=100,
    k=1,
    gamma_att=0.37,
    gamma_ali=0.2,
    gamma_rand=0.2,
    l_att=3.0,
    l_ali=3.0,
    tau_0=0.8,
    tau_n_mean=1.0,
    l_n_mean=1.0,
    eta=0.8,
    
    # Extension Parameters to Mimic Original Model
    # Make the burst phase nearly instantaneous
    # A very small, non-zero duty cycle
    omega=1e-10,
    # Only one decision instant per "burst"      
    n_omega=1,
)

start = time.time()
run_batch_simulation(
    initial_state_ext,
    rec_extended := SimulationRecorderExtended(skip_first_n=1000),
    steps=50000,
)
end = time.time()

print("Extended Model (Approximation) Results")
# The recorder for the extended model accumulates totals, so we average them
num_samples = rec_extended.total_samples - rec_extended.skip_first_n
avg_dispersion = rec_extended.total_dispersion / num_samples if num_samples > 0 else 0
avg_polarization = rec_extended.total_polarization / num_samples if num_samples > 0 else 0
avg_milling = rec_extended.total_milling / num_samples if num_samples > 0 else 0

print(f"Avg Dispersion: {avg_dispersion}")
print(f"Avg Polarization: {avg_polarization}")
print(f"Avg Milling: {avg_milling}")
print(f"Batch simulation time: {end - start} seconds")

Extended Model (Approximation) Results
Avg Dispersion: 64.00455584564473
Avg Polarization: 0.12267956320283672
Avg Milling: 0.013130864392895783
Batch simulation time: 7.901408433914185 seconds


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cb25d.batch import run_batch_simulation
from cb25d.simulation_impl_ext import generate_extended_initial_conditions, SimulationRecorderExtended
# Test different omega values

omega_values = [0.1, 0.3, 0.5, 0.7, 0.9]
results = []

for w in omega_values:
    print(f"Testing Omega = {w}...")
    # Initialize
    state = generate_extended_initial_conditions(
        seed=100, 
        n=50, 
        l_att=3.0, 
        omega=w,
        n_omega=5, 
        gamma_att=0.22,
        gamma_ali=0.6
    )
    
    # Record
    rec = SimulationRecorderExtended(skip_first_n=2000)
    run_batch_simulation(state, rec, steps=5000)
    results.append(rec)

# Plot polarization vs omega
plt.plot(omega_values, [r.total_polarization for r in results], marker='o')
plt.title("Effect of Duty Cycle on Schooling Stability")
plt.xlabel("Duty Cycle (omega)")
plt.ylabel("Group Polarization")
plt.grid(True)
plt.show()

# Plot dispersion vs omega
plt.plot(omega_values, [r.total_dispersion for r in results], marker='o', color='orange')
plt.title("Effect of Duty Cycle on Group Dispersion")
plt.xlabel("Duty Cycle (omega)")
plt.ylabel("Group Dispersion")
plt.grid(True)

# Plot milling vs omega
plt.plot(omega_values, [r.total_milling for r in results], marker='o', color='green')
plt.title("Effect of Duty Cycle on Milling Behavior")
plt.xlabel("Duty Cycle (omega)")
plt.ylabel("Group Milling")
plt.grid(True)
plt.show()


In [ ]:
# Test different n_omega values
n_omega_values = [1, 3, 5, 7, 9]

for n_omega in n_omega_values:
    print(f"Testing n_omega = {n_omega}...")
    # Initialize
    state = generate_extended_initial_conditions(
        seed=100, 
        n=50, 
        l_att=3.0, 
        omega=0.5,
        n_omega=n_omega, 
        gamma_att=0.22,
        gamma_ali=0.6
    )
    
    # Record
    rec = SimulationRecorderExtended(skip_first_n=2000)
    run_batch_simulation(state, rec, steps=5000)
    results.append(rec)

# Plot polarization vs n_omega
plt.plot(n_omega_values, [r.total_polarization for r in results], marker='o')
plt.title("Effect of Decision Steps per Burst on Schooling Stability")
plt.xlabel("Decision Steps per Burst (n_omega)")
plt.ylabel("Group Polarization")
plt.grid(True)
plt.show()

# Plot dispersion vs omega
plt.plot(n_omega_values, [r.total_dispersion for r in results], marker='o', color='orange')
plt.title("Effect of Duty Cycle on Group Dispersion")
plt.xlabel("Duty Cycle (omega)")
plt.ylabel("Group Dispersion")
plt.grid(True)

# Plot milling vs omega
plt.plot(n_omega_values, [r.total_milling for r in results], marker='o', color='green')
plt.title("Effect of Duty Cycle on Milling Behavior")
plt.xlabel("Duty Cycle (omega)")
plt.ylabel("Group Milling")
plt.grid(True)
plt.show()



In [ ]:
# To replicate the "Schooling" phase from the paper
initial_state_schooling = generate_extended_initial_conditions(
    seed=42,
    n=100,        
    k=1,

    gamma_att=0.22,
    gamma_ali=0.6,
    
    l_att=3.0,
    l_ali=3.0,
    gamma_rand=0.2,
    tau_0=0.8,
    tau_n_mean=1.0,
    l_n_mean=1.0,
    eta=0.8,
    
    # Extension Parameters to Mimic Original Model
    # Make the burst phase nearly instantaneous
    omega=0.01,
    n_omega=1,
)

In [ ]:
# To replicate the "Milling" phase from the paper
initial_state_milling = generate_extended_initial_conditions(
    seed=42,
    n=100,
    k=1,

    gamma_att=0.37,
    gamma_ali=0.2,

    l_att=3.0,
    l_ali=3.0,
    gamma_rand=0.2,
    tau_0=0.8,
    eta=0.8,

    # Extension Parameters to Mimic Original Model
    # Make the burst phase nearly instantaneous
    omega=0.01,
    n_omega=1,
)

In [ ]:
# To replicate the "Swarming" phase from the paper
initial_state_swarming = generate_extended_initial_conditions(
    seed=42,
    n=100,
    k=1,
    
    gamma_att=0.6,
    gamma_ali=0.6,
    
    l_att=3.0,

    # Extension Parameters to Mimic Original Model
    # Make the burst phase nearly instantaneous
    omega=0.01,
    n_omega=1,
)